# 总线类设备驱动

设计总线类设备驱动的目的是实现设备和驱动代码的分离，使设备驱动代码和设备驱动代码之间相互独立

设备专门用来描述设备所占有的硬件资源

驱动和设备通过总线绑定后

驱动负责从设备中动态获取这些资源信息
当设备的资源改变后，只是设备改变而已，驱动不用作过多修改，驱动代码的通用性大大提高


## 总线类设备的代码编写的大体流程为：

> 总线代码

1. 先定义一个总线结构体

```c
struct bus_type mybus =   
{
	.name = "mybus",	//总线的名字
	.match = mybus_match,  //匹配函数
};
```
总线结构体的类型为struct bus_type，其详细的定义在include/linux/device.h中

2. 注册上面定义的总线结构体
   在mybus_init函数中调用bus_register

3. 定义一个匹配函数mybus_match
   在mybus_init中调用bus_register时，会调用mybus_match

4. 模块卸载时，调用bus_unregister

> 驱动代码

1. 定义一个驱动结构体
```c
struct device_driver mydrv = 
{
    .name  = "fsdev_drv",
    .bus   = &mybus,          //驱动结构体与总线结构体关联
    .probe = mydrv_probe,     //驱动匹配函数
    .remove= mydrv_remove,    //驱动卸载函数
};
```

2. 注册驱动结构体
   在mydrv_init中调用driver_register

3. 定义一个匹配函数mydrv_probe
   在mydrv_init中调用driver_register时，会调用mydrv_probe

4. 定义一个卸载函数mydrv_remove
   在mydrv_exit中调用driver_unregister，会调用mydrv_remove

5. 卸载驱动结构体
   在mydrv_exit中调用driver_unregister

> 设备代码

1. 定义描述设备信息的结构体，可以在头文件中定义好，在设备代码中初始化
```c
struct mydev_desc dev_infos = 
{  // 设备的信息
    .name  = "dev_test", // 设备的名字
    .irqno = 666,      // 设备的中断号
    .addr  = 0x20033000,  // 设备的地址
};
```

2. 定义一个设备结构体
```c
struct device mydev = {
    .init_name     = "fsdev_drv",  /* initial name of the device */
    .bus           = &mybus,        //设备结构体与总线结构体关联
    .release       = mydev_release,  //设备卸载函数
    .platform_data = &dev_infos,   //自定义数据
};
```

3. 注册设备结构体
   在mydev_init中调用device_register

4. 定义一个卸载函数mydev_release
   当设备卸载时，会调用mydev_release

5. 卸载设备结构体
   在mydev_exit中调用device_unregister

In [ ]:
//总线部分vbus.c
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/device.h>

static int vbus_match(struct device *dev, struct device_driver *drv)  
// 匹配函数,负责匹配设备与驱动，如果匹配成功，则返回1
{
	return 1;
}
i
static struct bus_type vbus = 
{  
	// 定义总线数据结构
	.name = "vbus",  //总线的名字是vbus
	.match = vbus_match,  //匹配函数
};

EXPORT_SYMBOL(vbus);

static int __init vbus_init(void)
{
	return bus_register(&vbus);  // 注册总线
}
static void __exit vbus_exit(void)
{
	bus_unregister(&vbus);
}
module_init(vbus_init);
module_exit(vbus_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A virtual bus");

In [ ]:
//驱动部分vdrv.c
//一旦在总线中完成了匹配，驱动就动态地从设备部分中获取设备信息，然后调用驱动的probe函数
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/device.h>

extern struct bus_type vbus;

static struct device_driver vdrv = {
	.name = "vdrv",
	.bus = &vbus,
};

static int __init vdrv_init(void)
{
	return driver_register(&vdrv);
}

static void __exit vdrv_exit(void)
{
	driver_unregister(&vdrv);
}

module_init(vdrv_init);
module_exit(vdrv_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A virtual device driver");

In [ ]:
//设备部分vdev.c
// 保存设备所占用的资源信息
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/device.h>

extern struct bus_type vbus;

static void vdev_release(struct device *dev)
{
}

static struct device vdev =   // 设备结构体
{
	.init_name = "vdev",   // 设备名称
	.bus = &vbus,           // 设备所属总线
	.release = vdev_release,  // 设备释放函数
};

static int __init vdev_init(void)
{
	return device_register(&vdev);  // 注册设备
}

static void __exit vdev_exit(void)
{
	device_unregister(&vdev);
}

module_init(vdev_init);
module_exit(vdev_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A virtual device");

In [ ]:
//mybus.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/device.h>

int mybus_match(struct device *dev, struct device_driver *drv)
{
	//如果匹配成功，match方法一定要返回一个1， 失败返回0
	//因为是自己定义的总线，这里可以定义一个手动名称匹配的办法
	//先取出dev与drv的name
   	//不能直接使用dev->init_name，因为会把init_name赋给父类kobject,然后置空
	if(!strncasecmp(drv->name, dev->kobj.name, strlen(drv->name)))
	{
		printk("match ok\n");
		return 1;
	}
	else
	{
		printk("match failed\n");
		return 0;
	}    	
}

//实例化一个bus对象,用来管理device和driver，完成匹配
struct bus_type mybus = 
{
	.name = "mybus",	//总线的名字
	.match = mybus_match,  //匹配函数
};


EXPORT_SYMBOL(mybus);  //导出mybus,以便其他模块使用

static int __init mybus_init(void)
{
    printk("----------%s-------------\n", __FUNCTION__);

	//构建一个总线
	// /sys/bus/mybus
	int ret = bus_register(&mybus);  //注册总线
	if(ret != 0)
	{
		printk("bus_register error\n");
		return ret;
	}

	return 0;
}


static void __exit mybus_exit(void)
{
	printk("----------%s-------------\n", __FUNCTION__);

	bus_unregister(&mybus);
}

module_init(mybus_init);
module_exit(mybus_exit);

MODULE_LICENSE("GPL");

In [ ]:
//mydrv.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/device.h>
#include <linux/io.h>
#include "dev_info.h"

struct mydev_desc *pdesc;

//probe中设备相关数据来自struct device *dev
int mydrv_probe (struct device *dev)
{
    printk("--------------%s-------------\n",__FUNCTION__);

    pdesc = (struct mydev_desc *)dev->platform_data;

    printk("name  = %s\n", pdesc->name);
    printk("irqno = %d\n", pdesc->irqno);

    //假设要执行硬件相关操作
    unsigned long *paddr = ioremap(pdesc->addr, 8);
    
    return 0;
}

int mydrv_remove (struct device *dev)
{
    printk("--------------%s-------------\n",__FUNCTION__);
    return 0;
}

extern struct bus_type mybus;


struct device_driver mydrv = {

    .name  = "fsdev_drv",
    .bus   = &mybus,
    .probe = mydrv_probe,
    .remove= mydrv_remove,

};



static int __init mydrv_init(void)
{
    printk("--------------%s-------------\n",__FUNCTION__);

    //将驱动注册到总线中
    int ret;
    ret = driver_register(&mydrv);
    if(ret < 0)
    {
        printk("driver register failed\n");
        return ret;
    }

    return 0;
}

static void __exit mydrv_exit(void)
{
    printk("-------------%s------------\n",__FUNCTION__);
    driver_unregister(&mydrv);
}



module_init(mydrv_init);
module_exit(mydrv_exit);

MODULE_LICENSE("GPL");

In [ ]:
//dev_info.h
#ifndef __DEV_INFO_H__

#define _DEV_INFO_H__


//单独设置一个自定义数据，描述设备的特性
struct mydev_desc
{
    char *name;
    int irqno;
    unsigned long addr;
};

#endif

In [ ]:
#include <linux/init.h>
#include <linux/module.h>
#include <linux/device.h>

#include "dev_info.h"

extern struct bus_type mybus;

struct mydev_desc dev_infos = {  // 设备的信息
    .name  = "dev_test", // 设备的名字
    .irqno = 666,      // 设备的中断号
    .addr  = 0x20033000,  // 设备的地址
};

void mydev_release(struct device *dev)
{
    printk("------------%s-----------\n",__FUNCTION__);
}

//构建一个device对象
struct device mydev = {
    .init_name     = "fsdev_drv",  /* initial name of the device */
    .bus           = &mybus,
    .release       = mydev_release,
    .platform_data = &dev_infos,   //自定义数据
};

static int __init mydev_init(void)
{
    printk("------------%s-----------\n",__FUNCTION__);
    int ret;
    //将device注册到总线中去
    ret = device_register(&mydev);
    if(ret < 0)
    {
        printk("device_register failed\n");
        return ret;
    }

    return 0;
}

static int __exit mydev_exit(void)
{
    device_unregister(&mydev);

}


module_init(mydev_init);
module_exit(mydev_exit);

MODULE_LICENSE("GPL");


## 总线模块挂载后，在sys文件系统中会生成相应目录

![alt text](image-3.png)

当加载了vbus模块后

+ /sys/bus目录下会自动创建/sys/bus/vbus模块
+ /sys/bus/vbus目录下会自动创建/sys/bus/vbus/device目录
+ /sys/bus/vbus目录下会自动创建/sys/bus/vbus/driver目录

当加载了vdrv模块后

+ /sys/bus/vbus/drivers目录下自动创建/sys/bus/vbus/drivers/vdrv目录

当加载了vdev模块后

+ /sys/bus/vbus/deivce目录下自动创建/sys/bus/vus/device/vdev目录
+ /sys/deivce目录下也自动创建/sys/deivce/vdev目录

## 总线有那几种类型
1. 平台总线plaform bus
2. IIC总线
3. SPI总线
4. USB总线
5. PCI总线

# platform 总线